In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re

### ここだけ手動で設定

In [2]:
subject_name_list = ["kumakura","kim","souma","fujii","tubota","toki"]

In [4]:
pathes = glob.glob("/Users/miyakooti/repositories/arai_MATLAB_program/csv/?_*/HRV_and_PLI.csv")
pathes.sort()
pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/4_tubota/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/5_toki/HRV_and_PLI.csv']

In [5]:
hrv_df = pd.read_csv(f"{HRV_pathes[0]}")
hrv_df

,Unnamed: 0,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,...,vlf,lf,hf,lf/hf,p_total,vlf_perc,lf_perc,hf_perc,lf_nu,hf_nu
0,rest,73.678878,814.344647,40.280560,20.440067,36.813667,0.617801,0.196335,30,26.031042,...,363.483291,206.384215,482.573993,0.427674,1052.441499,34.537149,19.610042,45.852809,29.955985,70.044015
1,practice,75.523947,794.450000,31.370784,17.631999,31.337680,0.534050,0.082437,22,22.158335,...,649.675157,351.846586,207.483280,1.695783,1209.005023,53.736349,29.102161,17.161490,62.905024,37.094976
2,boredom,75.368816,796.085213,41.400093,17.968812,33.557870,0.610553,0.130653,24,23.728891,...,232.041729,290.761023,334.102552,0.870275,856.905303,27.079040,33.931523,38.989437,46.531921,53.468079
3,flow,79.462990,755.068493,45.496886,17.074905,30.303453,0.552369,0.093516,30,21.427739,...,744.117782,227.008970,72.257397,3.141671,1043.384149,71.317720,21.756989,6.925292,75.855156,24.144844
4,ultra,80.444654,745.854406,64.168076,19.123863,31.929731,0.526923,0.130769,56,22.573323,...,1666.610672,311.682250,119.862745,2.600326,2098.155668,79.432175,14.855058,5.712767,72.224740,27.775260
5,overload,75.991495,789.562044,39.822757,19.241337,33.166542,0.551220,0.124390,30,23.452286,...,682.892676,308.792854,115.237706,2.679616,1106.923235,61.692867,27.896501,10.410632,72.823255,27.176745


In [7]:
for i in range(0, int(len(HRV_pathes))):

    pli_path = PLI_pathes[i]
    pli_df = pd.read_csv(f"{pli_path}",index_col=0)
    
    hrv_path = HRV_pathes[i]
    hrv_df = pd.read_csv(f"{hrv_path}",index_col=0)
    
    df_concatted = pd.concat([hrv_df,pli_df], axis=1) # 横（axis=1）連結(concat)    
    df_concatted.to_csv(f"/Users/miyakooti/repositories/arai_MATLAB_program/csv/{i}_{subject_name_list[i]}/HRV_and_PLI.csv")

# ↓ sandbox
esc, enterで青と緑の切り替え
青状態のときにMでマークダウン

# [pandas参考文献](https://qiita.com/ysdyt/items/9ccca82fc5b504e7913a)

In [8]:
hrv_df[["lf","hf","lf/hf"]]

,lf,hf,lf/hf
rest,914.647545,135.422929,6.754008
practice,475.521521,87.387850,5.441506
boredom,1266.514349,184.554081,6.862565
flow,449.168624,160.526534,2.798096
ultra,203.494821,55.070206,3.695189
overload,299.521833,85.483935,3.503838


In [9]:
hrv_df.loc[0]

KeyError: 0

### 左のヘッダーはインデックスというらしい
https://note.nkmk.me/python-pandas-read-csv-tsv/

In [ ]:
hrv_df = pd.read_csv(f"{HRV_pathes[0]}", index_col=0)
hrv_df

In [ ]:
pli_df = pd.read_csv(f"{PLI_pathes[0]}", index_col=0)
pli_df

In [ ]:
df_concatted = pd.concat([hrv_df,pli_df], axis=1)
df_concatted

In [ ]:
# df_concatted.o_csv("/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/HRV_and_PLI.csv")

### [右結合とか左結合とかがわかるリンク](https://ai-inter1.com/pandas-dataframe_join/#st-toc-h-1)
- 連結はconcat
    - ただただガッちゃんこする
- 結合はmerge
    - 共通キーを指定して結合する